In [8]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd


driver_path = 'D:\Info Retrieval\chromedriver.exe'
chrome_service = webdriver.ChromeService(executable_path=driver_path)
driver=webdriver.Chrome(service=chrome_service)

company_links = []
companies = {}

target_url = "https://www.glassdoor.com/Explore/browse-companies.htm?industry=10012&country=SG&overall_rating_low=2.5&page="
num_pages = 1 # Number of pages to scrape -- CHANGE THIS TO THE AMOUNT REQUIRED

nums = list(range(1, num_pages+1))
url_mains = list(map(lambda n: target_url + str(n), nums))

count = 0

for url in url_mains:
    count += 1
    driver.get(url)
    time.sleep(3)
    
    elems = driver.find_elements(By.CLASS_NAME, "col-md-8")
    # print(elems)
    for elem in elems:
        # print("Text:", elem.text)
        inner_html = elem.get_attribute('innerHTML')
        # print("Secondary:", inner_html)

        soup = BeautifulSoup(inner_html, 'html.parser')

        # Add Company to dictionary
        outer_divs = soup.find_all('div', class_='mt-0 mb-std p-std css-mdw3bo css-errlgf')
        for outer_div in outer_divs:
            # Retrieve the value of the eid attribute
            eid_value = outer_div.get('data-brandviews').split('=')[-1]

            # Retrieve the value of employer id of the company
            print("Value of eid attribute:", eid_value)
            
            company_name = outer_div.find('h2').text
            print("Company Name:", company_name)
            companies[company_name] = eid_value

        print(companies)
    # response = driver.page_source
    if count % 3 == 0:
        driver.close()
        driver=webdriver.Chrome(service=chrome_service)
driver.close()

# Save the dictionary to a CSV file
df = pd.DataFrame(list(companies.items()),columns = ['Company Name','Employer ID'])
df.to_csv('companies.csv', index=False, encoding='utf-8')

Value of eid attribute: 6036
Company Name: Amazon
Value of eid attribute: 2763
Company Name: Deloitte
Value of eid attribute: 715
Company Name: Walmart
Value of eid attribute: 432
Company Name: McDonald's
Value of eid attribute: 194
Company Name: Target
Value of eid attribute: 7927
Company Name: Infosys
Value of eid attribute: 4138
Company Name: Accenture
Value of eid attribute: 354
Company Name: IBM
Value of eid attribute: 1651
Company Name: Microsoft
Value of eid attribute: 2202
Company Name: Starbucks
{'Amazon': '6036', 'Deloitte': '2763', 'Walmart': '715', "McDonald's": '432', 'Target': '194', 'Infosys': '7927', 'Accenture': '4138', 'IBM': '354', 'Microsoft': '1651', 'Starbucks': '2202'}


: 

In [5]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd
import random

def get_company_links(company_name:str, eid:int, page:int=1):
    return f"https://www.glassdoor.com/Reviews/{company_name}-Reviews-E{eid}_P{page}.htm?filter.iso3Language=eng"

# Initialize the Chrome driver
driver_path = 'D:\Info Retrieval\chromedriver.exe'
chrome_service = webdriver.ChromeService(executable_path=driver_path)
driver=webdriver.Chrome(service=chrome_service)

try:
    companies.items()
except:
    df = pd.read_csv('companies.csv')
    companies = dict(zip(df['Company Name'], df['Employer ID']))

for key, value in companies.items():
    # Initialize lists to store the data
    OverallRatings = []
    ReviewDates = []
    ReviewTitles = []
    JobTitles = []
    JobDetails = []
    Locations = []
    Pros = []
    Cons = []

    max_pages = 2
    for page in range(1, max_pages+1):

        url = get_company_links(key, value, page)
        print(url)
        print("Company Name:", key, "EID:", value, "Page:", page)
        driver.get(url)
        timer = random.randint(5, 10)
        time.sleep(timer)
        
        elem = driver.find_elements(By.ID, "ReviewsRef")[0]

        # print("Text:", elem.text)
        inner_html = elem.get_attribute('innerHTML')
        # print("Secondary:", inner_html)

        soup = BeautifulSoup(inner_html, 'html.parser')

        # print(soup.prettify())
        # Find the list item with class 'noBorder'
        list_items = soup.find_all('li', class_='noBorder')

        for list_item in list_items:

            # Retrieve the overall rating
            overallRating = list_item.find('span', class_="review-details__review-details-module__overallRating").text
            OverallRatings.append(overallRating)
            # print("OverallRating:", overallRating)

            # Retrieve the review date
            reviewDate = list_item.find('span', class_="review-details__review-details-module__reviewDate").text
            ReviewDates.append(reviewDate)
            # print("ReviewDate:", reviewDate)

            # Retrieve the review title
            reviewTitle = list_item.find('a', class_="review-details__review-details-module__detailsLink review-details__review-details-module__title").text
            ReviewTitles.append(reviewTitle)
            # print("ReviewTitle:", reviewTitle)

            # Retrieve the reviewer's job title
            jobTitle = list_item.find('span', class_="review-details__review-details-module__employee").text
            JobTitles.append(jobTitle)
            # print("JobTitle:", jobTitle)

            # Retrieve the reviewer's job details
            jobDetails = list_item.find('div', class_="review-details__review-details-module__employeeDetails").text
            JobDetails.append(jobDetails)
            # print("JobDetails:", jobDetails)

            # Retrieve the reviewer's location
            try:
                location = list_item.find('span', class_="review-details__review-details-module__location").text
            except:
                location = None
            Locations.append(location)
            # print("Location:", location)

            # Retrieve the review pros
            pros_and_cons = list_item.find_all('p', class_="review-details__review-details-module__isCollapsed")
            if len(pros_and_cons) < 2:
                pros_and_cons = list_item.find_all('p', class_="review-details__review-details-module__isExpanded")
            pros = pros_and_cons[0].text
            Pros.append(pros)
            # print("Pros:", pros)

            # Retrieve the review cons
            cons = pros_and_cons[1].text
            Cons.append(cons)
            # print("Cons:", cons)
        
            # print(list_item.prettify() )
            # reviews_feed = soup.find_all('div', id='ReviewsFeed')
            # print(len(reviews_feed.find('li', class_='noBorder')))

    # response = driver.page_source
            if page % 3 == 0:
                driver.close()
                driver=webdriver.Chrome(service=chrome_service)

    # response = driver.page_source
    # Put the data into the dataframe
    company_list = [key] * len(OverallRatings)
    Reviews = pd.DataFrame(list(zip(company_list, OverallRatings, ReviewDates, ReviewTitles, JobTitles, JobDetails, Locations, Pros, Cons)), 
                              columns = ['Company Name', 'Overall Rating', 'Review Date', 'Review Title', 'Job Title', 'Job Details', 'Location', 'Pros', 'Cons'])
    break

Reviews.to_csv('reviews.csv', index=False, encoding='utf-8')
driver.close()

https://www.glassdoor.com/Reviews/Amazon-Reviews-E6036_P1.htm?filter.iso3Language=eng
Company Name: Amazon
EID: 6036
https://www.glassdoor.com/Reviews/Amazon-Reviews-E6036_P2.htm?filter.iso3Language=eng
Company Name: Amazon
EID: 6036
